In [ ]:
! pip install -qU langchain-huggingface

In [ ]:
! pip install langchain-chroma


In [ ]:
! pip install -qU langchain langchain-community langchain-openai

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
text = [
    "篮球是一项伟大的运动。",
    "《带我飞向月球》是我最喜欢的歌曲之一。",
    "凯尔特人是我最喜欢的队伍。",
    "这是关于波士顿凯尔特人的一份文件。",
    "我真心喜欢去看电影。",
    "波士顿凯尔特人以20分的优势赢得了比赛。",
    "这只是一些随机的文字。",
    "《艾尔登法环》是过去15年里最棒的游戏之一。",
    "L. 科尔内特是凯尔特人队最优秀的球员之一。",
    "拉里·伯德是NBA的标志性球员。"
]

db = Chroma.from_texts(text, embeddings_model)


In [ ]:
query = "NBA的标志性球员是谁？"
retrieval = db.as_retriever(search_kwargs={"k": 10})
docs = retrieval.get_relevant_documents(query)
for doc in docs:
    print(f"- {doc.page_content}")

In [ ]:
from langchain_community.document_transformers import LongContextReorder
#对检索结果重新排序
reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(docs)
# Confirm that the 4 relevant documents are at beginning and end.
for doc in reordered_docs:
    print(f"- {doc.page_content}")

In [ ]:
import os
os.environ["DEEPSEEK_API_KEY"] = ""

In [ ]:
! pip install -qU langchain_deepseek

In [ ]:
from langchain_deepseek import ChatDeepSeek

llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt_template = """
Given these texts:
-----
{context}
-----
Please answer the following question:
{query}
"""
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "query"],
)


# Create and invoke the chain:
chain = create_stuff_documents_chain(llm, prompt)
response = chain.invoke({"context": reordered_docs, "query": query})
print(response)

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)
